In [1]:
import os
import pandas as pd
from rich import print as rprint

# Import util functions
from helper import set_openai_key,\
                    test_openai_api,\
                    create_openai_client,\
                    print_pretty,\
                    function_to_schema

# Import tools
from helper import generate_tree_string,\
                    get_lines_from_file,\
                    search_code, \
                    get_object_definition

print("Import successful!")

Import successful!


In [2]:
set_openai_key()
#test_openai_api()

API key set successfully.


In [3]:
# Define paths
comp_dir = "konwinski-prize"
comp_kaggle_evaluation_dir = os.path.join(comp_dir, "kaggle_evaluation")
comp_kprize_setup_dir = os.path.join(comp_dir, "kprize_setup")

comp_data_zip_path = os.path.join(comp_dir, "data.a_zip")
comp_data_dir = os.path.join(comp_dir, "data")
comp_data_parquet_path = os.path.join(comp_data_dir, "data.parquet")
comp_conda_packages_dir = os.path.join(comp_data_dir, "conda_packages")
comp_pip_packages_dir = os.path.join(comp_data_dir, "pip_packages")
comp_repo_configs_dir = os.path.join(comp_data_dir, "repo_configs")
comp_repos_dir = os.path.join(comp_data_dir, "repos")

In [4]:
# Load dataset
kprize_df = pd.read_parquet(comp_data_parquet_path)
kprize_df.head()

,instance_id,repo,problem_statement,patch,test_patch,pull_number,base_commit,PASS_TO_PASS,FAIL_TO_PASS,issue_numbers
0,pylint-dev__astroid-2496,pylint-dev/astroid,TypeError: unsupported format string passed to...,diff --git a/ChangeLog b/ChangeLog\nindex 4560...,diff --git a/tests/test_inference.py b/tests/t...,2496,8d3cdbbe6685fd8cf211816bec56c90f38f1859e,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::test_formatted_fstri...,[2492]
1,pylint-dev__astroid-2468,pylint-dev/astroid,Pylint checks against incorrect type with prop...,diff --git a/ChangeLog b/ChangeLog\nindex fdbb...,diff --git a/tests/test_inference.py b/tests/t...,2468,6db3a60553ff538a936d5dda23d67a3924a57f45,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::InferenceTest::test_...,[2467]
2,astropy__astropy-17048,astropy/astropy,QTable cannot take `dimensionless_unscaled` wh...,diff --git a/astropy/table/table.py b/astropy/...,diff --git a/astropy/table/tests/test_table.py...,17048,d60f6b72cd525262bfd179331d9fe4474177918f,[astropy/table/tests/test_table.py::TestSetTab...,[astropy/table/tests/test_table.py::test_qtabl...,[17047]
3,astropy__astropy-16898,astropy/astropy,BUG: tables do not deal well with zero-sized s...,diff --git a/astropy/io/registry/core.py b/ast...,diff --git a/astropy/io/fits/tests/test_connec...,16898,ee6d087baf301c1d08db92e6e5b6d909d57e6fac,[astropy/io/fits/tests/test_connect.py::TestSi...,[astropy/io/fits/tests/test_connect.py::test_z...,[16897]
4,astropy__astropy-16830,astropy/astropy,KeyError: 'version_1_3_or_later' when parsing ...,diff --git a/astropy/io/votable/tree.py b/astr...,diff --git a/astropy/io/votable/tests/test_tre...,16830,e39f486fec48d87aa3677326167954370d7a7bf9,[astropy/io/votable/tests/test_tree.py::test_c...,[astropy/io/votable/tests/test_tree.py::test_v...,"[16825, 16826]"


In [5]:
idx = 2
row = kprize_df.iloc[idx]
problem_statement = row["problem_statement"]
instance_id = row["instance_id"]
p2p_tests = row["PASS_TO_PASS"]
f2p_tests = row["FAIL_TO_PASS"]
repo_path = os.path.join(comp_repos_dir, f'repo__{instance_id}')

### Util Functions

In [6]:
# Generate function json schema
def example_function(a: int, b: str = "default", c: bool = True):
    """Example function with defaults."""
    pass
    
function_to_schema(example_function)

{'type': 'function',
 'function': {'name': 'example_function',
  'description': 'Example function with defaults.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer'},
    'b': {'type': 'string'},
    'c': {'type': 'boolean'}},
   'required': ['a']}}}

### Tools

#### Search & Discovery Tools

In [7]:
# Display tree
tree = generate_tree_string(repo_path)
rprint(tree[:1000])

├── CHANGES.rst
├── CITATION
├── CITATION.cff
├── CODE_OF_CONDUCT.md
├── CONTRIBUTING.md
├── GOVERNANCE.md
├── LICENSE.rst
├── MANIFEST.in
├── README.rst
├── SECURITY.md
├── astropy
│   ├── CITATION
│   ├── __init__.py
│   ├── _dev
│   │   ├── __init__.py
│   │   └── scm_version.py
│   ├── config
│   │   ├── __init__.py
│   │   ├── configuration.py
│   │   ├── paths.py
│   │   └── tests
│   │       ├── __init__.py
│   │       ├── data
│   │       │   ├── alias.cfg
│   │       │   ├── deprecated.cfg
│   │       │   ├── empty.cfg
│   │       │   └── not_empty.cfg
│   │       └── test_configs.py
│   ├── conftest.py
│   ├── constants
│   │   ├── __init__.py
│   │   ├── astropyconst13.py
│   │   ├── astropyconst20.py
│   │   ├── astropyconst40.py
│   │   ├── cgs.py
│   │   ├── codata2010.py
│   │   ├── codata2014.py
│   │   ├── codata2018.py
│   │   ├── config.py
│   │   ├── constant.py
│   │   ├── iau2012.py
│   │   ├── iau2015.py
│   │   ├── si.py
│   │   ├── tests
│   │   │   ├── __init_

In [8]:
print_pretty(function_to_schema(generate_tree_string))

{
    "function": {
        "description": "Generate a string representation of a directory tree structure.\n    \n    This function recursively traverses a directory and creates an ASCII tree visualization\n    similar to the Unix 'tree' command. It uses box-drawing characters to show the \n    hierarchy of files and directories.\n    \n    Args:\n        folder_path (str): Path to the directory to visualize\n        prefix (str, optional): Prefix string used for recursive indentation. \n                              Should not be set by user - used internally for recursion.\n                              Defaults to \"\".\n    \n    Returns:\n        str: A formatted string showing the directory tree structure using ASCII characters:\n            \u251c\u2500\u2500 for items that have siblings below them\n            \u2514\u2500\u2500 for the last item in a directory\n            \u2502   for vertical lines showing hierarchy\n    \n    Example:\n        >>> print(generate_tree_strin

In [9]:
# Get code snippet from line range
res = get_lines_from_file(file_path = os.path.join(repo_path, "astropy/_dev/scm_version.py"),
                                           line_range="1-20")
print(res)

# Try to use setuptools_scm to get the current version; this is only used
# in development installations from the git repository.
from pathlib import Path

try:
    from setuptools_scm import get_version

    version = get_version(root=Path(__file__).parents[1], relative_to=__file__)
except Exception:
    raise ImportError("setuptools_scm broken or not installed")



In [10]:
print_pretty(function_to_schema(get_lines_from_file))

{
    "function": {
        "description": "Returns the lines of code from a specified file and line range.\n\n    Args:\n        file_path (str):\n            The path to the file from which to retrieve lines.\n        line_range (str, optional):\n            A string representing the range of lines in the format \"start-end\".\n            For example, \"10-20\" means lines 10 through 20, inclusive (1-based indexing).\n            When not provided all lines are returned.\n        as_list (bool, optional):\n            Keep the lines separate.\n\n    Returns:\n        str:\n            A concatenated string of lines from the specified range. If the file\n            is not found or if the line range is invalid, returns an error message string.\n    \n    Example:\n        code = get_lines_from_file(file_path = os.path.join(\"astropy/_dev/scm_version.py\"),\n                                           line_range=\"1-20\")\n    \n    Output:\n        # Try to use setuptools_scm to get t

In [11]:
# Search code by string
res = search_code(root_directory=repo_path,
    search_string="normalize_kernel",
    n_lines_before=2,
    n_lines_after=2,
    return_imports=True
)

print_pretty(res[:2])

[
    {
        "content": "                image, psf, boundary=\"extend\", normalize_kernel=True",
        "context_after": [
            "            )",
            "            randomImage = np.random.normal(0, 2, image.shape)"
        ],
        "context_before": [
            "        def convolve_with_noise(image, psf):",
            "            convolvedImage = convolve("
        ],
        "file": "konwinski-prize/data/repos/repo__astropy__astropy-17048/astropy/visualization/tests/test_lupton_rgb.py",
        "imports": [
            "from astropy.convolution import Gaussian2DKernel, convolve",
            "from astropy.utils.compat.optional_deps import HAS_MATPLOTLIB",
            "from astropy.visualization import lupton_rgb",
            "from astropy.visualization.interval import ManualInterval",
            "from astropy.visualization.stretch import LinearStretch",
            "from numpy.testing import assert_allclose, assert_equal",
            "import matplotlib.pypl

In [12]:
print_pretty(function_to_schema(search_code))

{
    "function": {
        "description": "Searches for a given string in all .py files under root_directory.\n    \n    Optionally returning surrounding lines (context) and import statements from matching files.\n\n    Args:\n        root_directory (str): Path to the root directory of the codebase to search.\n        search_string (str): The string to search for in .py files.\n        n_lines_before (int, optional): Number of lines of context before match. Defaults to 0.\n        n_lines_after (int, optional): Number of lines of context after match. Defaults to 0.\n        return_imports (bool, optional): Whether to return import statements. Defaults to False.\n\n    Returns:\n        List[Dict[str, Union[str, int, List[str]]]]: List of match dictionaries.",
        "name": "search_code",
        "parameters": {
            "properties": {
                "n_lines_after": {
                    "type": "integer"
                },
                "n_lines_before": {
                  

In [13]:
# Get Class definition
res = get_object_definition(
    root_directory=repo_path,
    object_name="Gaussian1DKernel",
    return_imports=True
)

print_pretty(res)

{
    "content": "class Gaussian1DKernel(Kernel1D):",
    "definition_block": [
        "class Gaussian1DKernel(Kernel1D):",
        "    \"\"\"",
        "    1D Gaussian filter kernel.",
        "",
        "    The Gaussian filter is a filter with great smoothing properties. It is",
        "    isotropic and does not produce artifacts.",
        "",
        "    The generated kernel is normalized so that it integrates to 1.",
        "",
        "    Parameters",
        "    ----------",
        "    stddev : number",
        "        Standard deviation of the Gaussian kernel.",
        "    x_size : int, optional",
        "        Size of the kernel array. Default = \u230a8*stddev+1\u230b.",
        "    mode : {'center', 'linear_interp', 'oversample', 'integrate'}, optional",
        "        One of the following discretization modes:",
        "            * 'center' (default)",
        "                Discretize model by taking the value",
        "                at the cen

In [14]:
print_pretty(function_to_schema(get_object_definition))

{
    "function": {
        "description": "Searches the codebase for the first definition of a function, class, or method matching object_name.\n\n    If object_name is a method referenced with dot notation (e.g. \"Cat._meow\"),\n    then we find class 'Cat', extract the relevant portion of its definition block,\n    and include the method definition plus any __init__.\n\n    Args:\n        root_directory (str):\n            The path to the root directory of the codebase.\n        object_name (str):\n            The name of the function or class to find (e.g., \"my_function\", \"MyClass\", or \"Cat._meow\").\n        return_imports (bool, optional):\n            Whether to collect import statements found in the file.\n\n    Returns:\n        Optional[Dict[str, Union[str, int, List[str]]]]: \n            A dictionary describing the object definition, or None if not found.\n                - file (str): Path to the file containing the definition.\n                - line (int): The 1-bas